# 0. Initialize

Google Driveのマウント、モジュールのインポート、パラメータの設定等を行います。

**※ テスト用のFITSをfitsget.ipynbで取得している必要があります。**また、作業ディレクトリを変数workdirで指定するので、適宜設定して下さい。

In [0]:
# 適宜設定すべし
workdir = '/content/drive/My Drive/test'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
import os
from glob import iglob

from astropy.io import fits
from astropy.wcs import WCS
import cupy  as cp
import numpy as np

import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval

In [0]:
for f in iglob('/content/drive/**/eclair.py',recursive=True):
  eclairpath = os.path.dirname(f)
  break
  
sys.path.append(eclairpath)

from eclair import FitsContainer, reduction, fixpix, imalign, imcombine

In [0]:
fitslist = ['raw%02d.fits'%i for i in range(10)]
# 枚数が多い場合のベンチマークは以下のように擬似的に枚数を増やして行います
# fitslist *= 10
# 増やしすぎるとVRAMがオーバーフローします

dark = 'dark.fits'
flat = 'flat.fits'
bpmask = 'bpmask.fits'

output = 'combine.fits'

In [0]:
xy0 = np.array([[450,450],[450,650],[650,450],[650,650]])
slices = dict(xmin=52, xmax=1072, ymin=2, ymax =1022)

In [0]:
os.chdir(workdir)

  # 1. Data Loading
  eclair.FitsContainerクラスを利用して一次処理を行う天体画像を読み込みます。
  loadメソッドを実行すると、与えられたリスト内のFITSをカレントディレクトリから探して読み込みます。
  読み込んだ情報のうち、ヘッダはFitsContainer.headerにFITS名とastropy.io.fits.headerのdictとして、画像データはFitsContainer.dataに3次元cupy.ndarrayとして格納されます。dataは第2軸がY軸、第3軸がX軸に対応し、各画像が第1軸方向にスタックされています。
  
  ※ 用意した画像はオーバースキャン領域が含まれるため、setsliceメソッドでスライス範囲を設定しています。

In [0]:
fc = FitsContainer(fitslist)
fc.setslice(**slices)
fc.load()

In [0]:
print(type(fc.header), type(fc.data))

In [0]:
npdark = fits.getdata(dark).astype('f4')
npflat = fits.getdata(flat).astype('f4')
npbpm = fits.getdata(bpmask).astype('f4')

cpdark = cp.array(npdark)
cpflat = cp.array(npflat+(npflat==0.0).astype('f4'))
cpbpm = cp.sign(cp.array(npbpm))

# 2. Get shift
用意したFITSにはWCSが貼り付けてあるので、これを利用して画像同士の相対位置を求めます。

In [0]:
wcs = {f:WCS(fc.header[f]) for f in fitslist}

In [0]:
def getshift(idx):
  ad0 = wcs[fitslist[idx]].wcs_pix2world(xy0,1)
  shift = np.empty([len(fitslist),2],dtype='f4')
  for i, f in enumerate(fitslist):
    xy = wcs[f].wcs_world2pix(ad0,1)
    shift[i,...] = (xy0-xy).mean(axis=0)
    
  return shift

In [0]:
shift = getshift(0)
idx = np.linalg.norm(shift-np.median(shift,axis=0),axis=1).argmin()
shift = getshift(idx)

# 3. Reduction
まず、eclair.reduction関数でバイアス、ダーク引き、フラット補正を一度に行います。この関数は、この例で言えば
```
fc.data = (fc.data - cpbias - cpdark) / cpflat
```
という式と同じですが、関数呼び出しのオーバーヘッドとメモリ使用量がより少ないです。各入力値はbroadcastableである必要があります。(reshapeでcpbiasの次元を増しているのはこのためです。)

次に、eclair.fixpix関数でバッドピクセルの補正を行います。この関数はバッドピクセルを周囲のピクセルカウントの平均で上書きします。与えるバッドピクセルマスクは、バッドピクセルの位置の値が1、それ以外が0である必要があります。

※ "%time"は実行時間計測のマジックコマンドで、コードの実行には関係ありません。

In [0]:
bias = [fc.header[f]['PEDLEVEL'] for f in fitslist]
cpbias = cp.array(bias,dtype='f4').reshape(-1,1,1)

In [0]:
%time fc.data = reduction(fc.data, cpbias, cpdark, cpflat)

In [0]:
%time fc.data = fixpix(fc.data, cpbpm)

# 4. Align
先程求めた相対位置をもとに、画像同士の位置合わせを行います。
shiftは2次元のnumpy.ndarrayで第2軸方向の1つ目の値がX座標、2つ目の値がY座標と解釈されます。
第1軸方向の並び順は、第1引数で与えるarrayに於ける画像の並び順と同じである必要があります。

サブピクセルシフト時の補間アルゴリズムのデフォルトは3次スプライン補間です。

In [0]:
%time fc.data = imalign(fc.data, shift)

# 5. Combine
画像の重ね合わせを行います。デフォルトのアルゴリズムはsigmaclippedmeanです。

In [0]:
%time imcombine(fitslist, fc.data, output, overwrite=True)

# 6. Show combined FITS

In [0]:
ls combine.fits

In [0]:
data = fits.getdata(output)
vrange = ZScaleInterval().get_limits(data)
data = data.clip(*vrange)
y_len, x_len = data.shape

plt.figure()
plt.imshow(data)
plt.xlim(0,x_len)
plt.ylim(0,y_len)
plt.colorbar()
plt.show()